In [1]:
print("Ok")

Ok


In [24]:
%pwd

'd:\\shruti(new)\\study\\Medical-Chatbot\\research'

In [25]:
import os
os.chdir("../")

In [26]:
%pwd

'd:\\shruti(new)\\study\\Medical-Chatbot'

In [22]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [27]:
# Extract Data from the PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents


In [28]:
extracted_data = load_pdf_file(data='Data/')

In [15]:
# extracted_data

In [18]:
# Split the data into Text Chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [29]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks:", len(text_chunks))

Length of Text Chunks: 5859


In [21]:
# text_chunks

In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings


In [6]:
# Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")
print("Loaded successfully")



c:\Users\Dell\miniconda3\envs\mchatbot310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded successfully


In [7]:
embeddings = download_hugging_face_embeddings()

C:\Users\Dell\AppData\Local\Temp\ipykernel_9420\2083297909.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')


In [8]:
query_result = embeddings.embed_query("Hello World")
print("Length:", len(query_result))

Length: 384


In [10]:
# query_result

In [47]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
COHERE_API_KEY=os.environ.get('COHERE_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-bot"

pc.create_index(
    name = index_name,
    dimension = 1024,
    metric = "cosine",
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'd487a61b6a02ae072d6f25c78303c7ac', 'date': 'Thu, 19 Jun 2025 12:33:43 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [52]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["COHERE_API_KEY"] = COHERE_API_KEY

In [30]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docSearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [53]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your pinecone index.
docSearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [32]:
docSearch

In [59]:
retriever = docSearch.as_retriever(search_type="similarity", search_kwags={"k":4})

In [60]:
retrieved_docs = retriever.invoke("What is Acne?")

In [35]:
retrieved_docs

[Document(id='f9c5bae5-f95a-4809-bfa9-412ec614f4c3', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='a15a103b-d822-43d3-8856-3c084a16fa02', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM 

In [66]:
from langchain_cohere import CohereEmbeddings, ChatCohere
COHERE_API_KEY = os.environ.get("COHERE_API_KEY")
os.environ["COHERE_API_KEY"] = COHERE_API_KEY
llm = ChatCohere(model="command-r",temperature=0.4, max_tokens=500)

In [85]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant that ONLY answers using the given context.\n"
    "If the context does not contain the answer, say:\n"
    "'I don't know based on the provided context.'\n\n"
    "Do NOT use your own knowledge.\n"
    "Context:\n{context}"
)



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [86]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [87]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

Acne is a common skin disease that causes pimples to break out on the face, chest, and back. The pores of the skin become clogged with oil, bacteria, and dead skin cells, leading to inflammation. It is also referred to as acne vulgaris, affecting nearly 17 million people in the United States.


In [88]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I don't know based on the provided context. The word 'stats' does not appear in the provided text.


In [94]:
response = rag_chain.invoke({"input": "how to cure cold?"})
print(response["answer"])

There are multiple ways to treat the common cold. Here are some of the recommended treatments:

1. Essential oils can be used to treat cold symptoms. Inhalation of eucalyptus oil or other essential oils in warm steam can help relieve congestion. Lavender and ginger oils can be used in hot or cold compresses to soothe muscle aches. For a sore throat, antiseptic oils like tea tree oil and sage can be mixed with water for gargling or as a mouthwash.

2. Herbal teas can provide relief. Herbalists recommend a tea blend made from mullein, coltsfoot, and anise seed.

3. Homeopathic medicine and traditional Chinese medicine may also be effective in treating cold symptoms.

4. Hydrotherapy can help clean the chest and boost the immune system's response.

5. For fever associated with a cold, acetaminophen (Tylenol) can be taken to reduce temperature.

6. Getting plenty of rest and drinking fluids are general recommendations to aid in recovery. Some people find hot liquids, especially warm teas, 

In [95]:
response = rag_chain.invoke({"input": "what is acromegaly and gigantism?"})
print(response["answer"])

Acromegaly and gigantism are disorders related to increased growth in the body. Acromegaly is a condition in which there is abnormal growth in bone and soft tissue due to the excessive release of a chemical, known as growth hormone (GH), from the pituitary gland in the brain. This disorder usually occurs in adults, leading to progressive bone growth and soft tissue swelling, particularly in the hands, feet, and face. 

Gigantism, on the other hand, is a similar but distinct condition, characterized by excessive growth hormone secretion during puberty, resulting in extreme height and skeletal growth. Gigantism is often associated with a benign tumor of the pituitary gland. 

Both disorders can have serious health consequences, including cardiovascular problems, joint pain, diabetes, and other complications.
